In [84]:
import nltk
from collections import defaultdict
import pandas as pd

In [85]:
from nltk.corpus import stopwords

In [86]:
from nltk.tokenize import TweetTokenizer

In [87]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shengruilyu/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [88]:
stop_words = set(stopwords.words('english'))

In [89]:
topk = 2

In [90]:
df = pd.read_csv("../regex_filtering/after_regex.csv", index_col = 0, low_memory = False)

In [91]:
tknzr = TweetTokenizer()

In [92]:
word_frequency_by_subreddit= {}

In [93]:
topic_postags = set(['NN', 'NNS', 'NNP', 'NNPS'])

In [94]:
for index, row in df.iterrows():
    sub_reddit = row['subreddit']
    if sub_reddit in word_frequency_by_subreddit:
        word_frequency = word_frequency_by_subreddit[sub_reddit]
    else:
        word_frequency = defaultdict(int)
        word_frequency_by_subreddit[sub_reddit] = word_frequency
    tokens = tknzr.tokenize(row['body'])
    tagged = nltk.pos_tag(tokens)
    for token,tag in tagged:
        if (token not in stop_words) and (token.isalnum()) and (tag in topic_postags):
            word_frequency[token]+=1
    

In [95]:
word_frequency_by_subreddit

{'books': defaultdict(int,
             {'pleasures': 1,
              'reply': 1,
              'speaks': 1,
              'writing': 1,
              'book': 16,
              'memorable': 1,
              'Huh': 1,
              'So': 1,
              'guy': 1,
              'someone': 3,
              'answer': 1,
              'Juice': 1,
              'Terry': 1,
              'Mark': 1,
              'Whatney': 1,
              'Martian': 1,
              'sense': 2,
              'humor': 1,
              'Mars': 1,
              'sure': 2,
              'Murakami': 1,
              'sub': 1,
              'Thank': 8,
              'recommendation': 3,
              'Laundry': 1,
              'history': 1,
              'fantasy': 1,
              'inside': 1,
              'jokes': 1,
              'allusions': 1,
              'alley': 1,
              'summary': 1,
              'series': 2,
              'audience': 1,
              'genre': 1,
              'character': 1

In [96]:
topic_per_by_subreddit = {}

In [97]:
for sub_reddit, word_frequency in word_frequency_by_subreddit.items():
    top_words = [k for k, v in sorted(word_frequency.items(), key=lambda item: item[1], reverse= True)]
    topics = set([word for word in top_words[:topk] if word_frequency[word]>1])
    topic_per_by_subreddit[sub_reddit] = topics

In [98]:
topic_per_by_subreddit

{'books': {'Thank', 'book'},
 'tifu': {'flight', 'someone'},
 'AskReddit': {'people', 'water'},
 'Awwducational': {'articles', 'species'},
 'personalfinance': {'TaxAct', 'money'},
 'AnimalsBeingBros': {'Svenn', 'time'},
 'aww': {'day', 'dog'},
 'news': {'people', 'police'},
 'dataisbeautiful': {'difference', 'job'},
 'trashy': {'man', 'people'},
 'WinStupidPrizes': {'bitch', 'people'},
 'TIHI': {'people', 'things'},
 'awfuleverything': {'year', 'years'},
 'therewasanattempt': {'attempt', 'diamonds'},
 'Unexpected': {'girl', 'guy'},
 'Cringetopia': {'couple', 'thing'},
 'funny': {'day', 'people'},
 'DunderMifflin': {'Kevin', 'Pam'},
 'crappyoffbrands': set(),
 'holdmycosmo': {'holdmyfeedingtube', 'way'},
 'KidsAreFuckingStupid': {'home', 'truck'},
 'wholesomememes': {'Credits', 'time'},
 'BikiniBottomTwitter': {'Spongebob', 'gif'},
 'memes': {'man', 'meme'},
 'Whatcouldgowrong': {'car', 'people'},
 'assholedesign': {'ads', 'sugar'},
 'disneyvacation': {'FUCK'},
 'ExpectationVsReality': 

In [100]:
for index, row in df.iterrows():
    sub_reddit = row['subreddit']
    tokens = tknzr.tokenize(row['body'])
    new_tokens = []
    for token in tokens:
        if token in topic_per_by_subreddit[sub_reddit]:
            new_tokens.append("TOPIC")
        else:
            new_tokens.append(token)
    row['body'] = " ".join(new_tokens)

In [101]:
df.head()

,body,author,subreddit,_replies,_submission,awarders,total_awards_received,approved_at_utc,link_title,mod_reason_by,...,subreddit_name_prefixed,controversiality,locked,author_flair_background_color,collapsed_because_crowd_control,mod_reports,quarantine,subreddit_type,ups,_fetched
0,"Oh , we all have those guilty pleasures ! But ...",dmur726,books,[],NaN,[],0,NaN,What fictional character have you met while re...,NaN,...,r/books,0,False,NaN,NaN,[],False,public,2,True
1,I agree with you The really sad thing is that ...,dmur726,tifu,[],NaN,[],0,NaN,"TIFU by calling the dog biscuits ""cookies""",NaN,...,r/tifu,0,False,NaN,NaN,[],False,public,3,True
2,"You are right , and we didn t even think about...",dmur726,tifu,[],NaN,[],0,NaN,"TIFU by calling the dog biscuits ""cookies""",NaN,...,r/tifu,0,False,NaN,NaN,[],False,public,3,True
3,Thank you ! She was one if those one in a life...,dmur726,tifu,[],NaN,[],0,NaN,"TIFU by calling the dog biscuits ""cookies""",NaN,...,r/tifu,0,False,NaN,NaN,[],False,public,2,True
4,Huh So he is that guy that you would like to b...,dmur726,books,[],NaN,[],0,NaN,What fictional character have you met while re...,NaN,...,r/books,0,False,NaN,NaN,[],False,public,2,True


In [102]:
df.to_csv('after_top' + str(topk) + '_removal.csv')